### Basic global settings trials including ncluster, method and data used

In [1]:
import os
import json
import itertools
import pandas as pd
from functions import *
%set_env GUROBI_HOME=/usr/local/gurobi/linux64

env: GUROBI_HOME=/usr/local/gurobi/linux64


In [2]:
with open('config.json', 'r') as f:
    config_all = json.loads(f.read())

config = config_all['Texas_7_years_1']
inputs = config['inputs']
settings = config['settings']
print(inputs)
pv_all = np.genfromtxt(
    os.path.join(config['root'], config['pv_path']), delimiter=',', filling_values=0)
wind_all = np.genfromtxt(
    os.path.join(config['root'], config['wind_path']), delimiter=',', filling_values=0)
load_all = np.genfromtxt(
    os.path.join(config['root'], config['load_path']), delimiter=',', filling_values=0)  # Data load

{'c_bat': 200000, 'c_bat_power': 70000, 'c_pv': 1000000, 'c_wind': 1500000, 'c_gen_inv': 1000000, 'c_gen_a': 0.1, 'c_gen_b': 30, 'up_time': 6, 'down_time': 6, 'energy_cap': 200, 'power_cap': 200, 'gen_cap': 0, 'mdc': 50, 'effi': 0.9, 'rps': 0.8, 'note_2': 'This property is set to indicate what are above properties for model. Change with model parameters.', 'solver': 1, 'print_log': 0, 'timelimit': 210240, 'note_3': 'This property is set to indicate what are above properties for solver. No need to change.'}


In [3]:
day_num = settings['day_num']
time_set = np.arange(24*day_num)  # Time horizon
if 'profile_id' in settings:
    profile_id = settings['profile_id']
    renewable = [wind_all[time_set, profile_id],pv_all[time_set, profile_id]]
else:
    # Denote that first profile is used and no other profile exists.
    profile_id = -1
    renewable = [wind_all[time_set], pv_all[time_set]]
load = load_all[time_set]*100  # Data extract

In [4]:
list_temp = itertools.product([100000,150000],[50000,30000],[500000,750000],[1000000,1250000])
backups = pd.DataFrame(columns=['c_bat','c_bat_power','c_pv','c_wind'],data=list_temp)
for backup in backups.index:
    print("Index :" + str(backup))
    update = backups.loc[backup,:].to_dict()
    for item in ['c_bat','c_bat_power','c_pv','c_wind']:
        inputs[item] = update[item]
    config['inputs'] = inputs
    expected = pd.read_csv('/home/jupyter-zyh/Gnhe/benchmark/Linear/Texas/profile-{}/expected/expected_2555_{}_1e-4.csv'.format(profile_id,backup), index_col=0).loc[0,:].to_dict()
    sim_features_df = pd.read_csv('/home/jupyter-zyh/Gnhe/benchmark/Linear/Texas/profile-{}/features/features_2555_{}_1e-4.csv'.format(profile_id,backup), index_col=0)
    df = run_trials(config, wind_all, pv_all, load_all, expected, sim_features_df[['renewable_cap_0','renewable_cap_1','max_energy','max_power']])
    df.set_index('description').to_csv(
        config['save_root']+settings['data']+('_Int_' if inputs['gen_cap'] else '_')+('Renewable'+str(profile_id)+'_' if 'profile_id' in settings else '_')+str(settings['day_num'])+'Days_'+str(settings['period'])+'HrsPerPeriod_{}.csv'.format(backup))
print("All Done!")

Index :0
1th trial:  2.1902074813842773
2th trial:  2.5414628982543945
3th trial:  1.7992584705352783
4th trial:  2.234307050704956
5th trial:  1.9901251792907715
6th trial:  2.2615952491760254
7th trial:  1.7904341220855713
8th trial:  2.1925854682922363
9th trial:  1.8157875537872314
10th trial:  2.273505449295044


IndexError: index 4 is out of bounds for axis 0 with size 4